In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\BigData\\spark'

In [2]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkFiles
sc = SparkContext()
sc.addFile("heart.csv")
sqlContext = SQLContext(sc)

Py4JJavaError: An error occurred while calling o19.addFile.
: java.io.FileNotFoundException: File file:/E:/2CS/Semester 2/BDM/Deep Learning 2/heart.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:666)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:987)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:656)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:454)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1604)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1580)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflec

In [ ]:
df = sqlContext.read.csv(SparkFiles.get("heart.csv"), header=True, inferSchema= True)

In [ ]:
df.printSchema()

In [ ]:
df.show(5, truncate = False)

In [ ]:
df.select('age', 'target').show()

In [ ]:
df.groupBy("target").count().sort("count",ascending=True).show()

In [ ]:
df.describe().show()

In [ ]:
df.describe('age').show()

In [ ]:
df.crosstab('age', 'target').sort("age_target").show()

In [ ]:
df.filter(df.age > 0).count()

In [ ]:
df.groupby('target').agg({'age': 'mean'}).show()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
CATE_FEATURES = ['age', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 
                 'thal']
stages = []
for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx =  StringIndexer(inputCol="target", outputCol="newtarget")
stages += [label_stringIdx]
assemblerInputs = [c + "classVec" for c in CATE_FEATURES]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df)
model = pipelineModel.transform(df)

In [ ]:
model.count()

In [ ]:
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["newtarget"], DenseVector(x["features"])))

In [ ]:
df_train = sqlContext.createDataFrame(input_data, ["target", "features"])

In [ ]:
df_train.count()

In [ ]:
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)

In [ ]:
train_data.groupby('target').agg({'target': 'count'}).show()

In [ ]:
test_data.groupby('target').agg({'target': 'count'}).show()

In [ ]:
# Import `LinearRegression`
from pyspark.ml.classification import LogisticRegression

# Initialize `lr`
lr = LogisticRegression(labelCol="target",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)

# Fit the data to the model
linearModel = lr.fit(train_data)

In [ ]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(linearModel.coefficients))
print("Intercept: " + str(linearModel.intercept))

In [ ]:
# Make predictions on test data using the transform() method.
predictions = linearModel.transform(test_data)

In [ ]:
predictions.printSchema()

In [ ]:
predictions.show()

In [ ]:
cm = predictions.select("target", "prediction")

In [ ]:
cm.groupby('target').agg({'target': 'count'}).show()

In [ ]:
cm.groupby('prediction').agg({'prediction': 'count'}).show()

In [ ]:
acc = (cm.filter(cm.target == cm.prediction).count() / cm.count())*100
print("Model accuracy: %.2f%%" % (acc))